<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Challenge-3" data-toc-modified-id="Challenge-3-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Challenge 3</a></span><ul class="toc-item"><li><span><a href="#Q1:-How-to-identify-VIP-&amp;-Preferred-Customers?" data-toc-modified-id="Q1:-How-to-identify-VIP-&amp;-Preferred-Customers?-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Q1: How to identify VIP &amp; Preferred Customers?</a></span></li><li><span><a href="#How-to-label-customers-whose-aggregated-amount_spent-is-in-a-given-quantile-range?" data-toc-modified-id="How-to-label-customers-whose-aggregated-amount_spent-is-in-a-given-quantile-range?-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>How to label customers whose aggregated <code>amount_spent</code> is in a given quantile range?</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Sub-Problem-1:-How-to-aggregate-the--amount_spent-for-unique-customers?" data-toc-modified-id="Sub-Problem-1:-How-to-aggregate-the--amount_spent-for-unique-customers?-1.2.0.1"><span class="toc-item-num">1.2.0.1&nbsp;&nbsp;</span>Sub Problem 1: How to aggregate the  <code>amount_spent</code> for unique customers?</a></span></li><li><span><a href="#Sub-Problem-2:-How-to-select-customers-whose-aggregated-amount_spent-is-in-a-given-quantile-range?" data-toc-modified-id="Sub-Problem-2:-How-to-select-customers-whose-aggregated-amount_spent-is-in-a-given-quantile-range?-1.2.0.2"><span class="toc-item-num">1.2.0.2&nbsp;&nbsp;</span>Sub Problem 2: How to select customers whose aggregated <code>amount_spent</code> is in a given quantile range?</a></span></li><li><span><a href="#Sub-Problem-3:-How-to-label-selected-customers-as-&quot;VIP&quot;-or-&quot;Preferred&quot;?" data-toc-modified-id="Sub-Problem-3:-How-to-label-selected-customers-as-&quot;VIP&quot;-or-&quot;Preferred&quot;?-1.2.0.3"><span class="toc-item-num">1.2.0.3&nbsp;&nbsp;</span>Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?</a></span></li></ul></li></ul></li><li><span><a href="#Q2:-How-to-identify-which-country-has-the-most-VIP-Customers?" data-toc-modified-id="Q2:-How-to-identify-which-country-has-the-most-VIP-Customers?-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Q2: How to identify which country has the most VIP Customers?</a></span></li><li><span><a href="#Q3:-How-to-identify-which-country-has-the-most-VIP+Preferred-Customers-combined?" data-toc-modified-id="Q3:-How-to-identify-which-country-has-the-most-VIP+Preferred-Customers-combined?-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Q3: How to identify which country has the most VIP+Preferred Customers combined?</a></span></li></ul></li></ul></div>

# Challenge 3

In this challenge we will work on the `Orders` data set (https://drive.google.com/file/d/1xUzMpyQgC5IaK61ehUH2C5P4jCPmLa-s/view?usp=sharing). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Data Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [122]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [123]:
orders = pd.read_csv("data/Orders.csv")
orders.columns
print(orders.shape)
orders.drop(labels=["Unnamed: 0"],axis=1,inplace=True)
orders.head()

(397924, 14)


,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [124]:
orders.describe()

,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,CustomerID,amount_spent
count,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000
mean,560617.126645,2010.934259,7.612537,3.614555,12.728247,13.021823,3.116174,15294.315171,22.394749
std,13106.167695,0.247829,3.416527,1.928274,2.273535,180.420210,22.096788,1713.169877,309.055588
min,536365.000000,2010.000000,1.000000,1.000000,6.000000,1.000000,0.000000,12346.000000,0.000000
25%,549234.000000,2011.000000,5.000000,2.000000,11.000000,2.000000,1.250000,13969.000000,4.680000
50%,561893.000000,2011.000000,8.000000,3.000000,13.000000,6.000000,1.950000,15159.000000,11.800000
75%,572090.000000,2011.000000,11.000000,5.000000,14.000000,12.000000,3.750000,16795.000000,19.800000
max,581587.000000,2011.000000,12.000000,7.000000,20.000000,80995.000000,8142.750000,18287.000000,168469.600000


In [125]:
total_amount_orders = orders.groupby('CustomerID').sum()
total_amount_orders.reset_index(inplace=True,drop=False)
total_amount_orders.rename(columns={"amount_spent":"total_amount"},inplace=True)
total_amount_orders.head()


C:\Users\marci\AppData\Local\Temp\ipykernel_28412\339314403.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_amount_orders = orders.groupby('CustomerID').sum()


,CustomerID,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,total_amount
0,12346,541431,2011,1,2,10,74215,1.04,77183.60
1,12347,101296926,365971,1383,441,2219,2458,481.21,4310.00
2,12348,16869685,62324,257,111,472,2341,178.71,1797.24
3,12349,42165457,146803,803,73,657,631,605.10,1757.55
4,12350,9231629,34187,34,51,272,197,65.30,334.40


In [126]:
total_amount_orders["Client_status"] = pd.qcut(total_amount_orders["total_amount"], q=[0, .75,.95,1], labels=["Rookie", "Preferred", "VIP"])

In [134]:
bins_total_amount = pd.qcut(total_amount_orders["total_amount"], q=[0, .75,.95,1], labels=["Rookie", "Preferred", "VIP"], retbins=True)
bins_total_amount

(0             VIP
 1       Preferred
 2       Preferred
 3       Preferred
 4          Rookie
           ...    
 4334       Rookie
 4335       Rookie
 4336       Rookie
 4337    Preferred
 4338    Preferred
 Name: total_amount, Length: 4339, dtype: category
 Categories (3, object): ['Rookie' < 'Preferred' < 'VIP'],
 array([     0.   ,   1661.64 ,   5840.182, 280206.02 ]))

In [127]:
total_amount_orders.head()

,CustomerID,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,total_amount,Client_status
0,12346,541431,2011,1,2,10,74215,1.04,77183.60,VIP
1,12347,101296926,365971,1383,441,2219,2458,481.21,4310.00,Preferred
2,12348,16869685,62324,257,111,472,2341,178.71,1797.24,Preferred
3,12349,42165457,146803,803,73,657,631,605.10,1757.55,Preferred
4,12350,9231629,34187,34,51,272,197,65.30,334.40,Rookie


In [128]:
orders_catclient = pd.merge(orders,total_amount_orders[["Client_status","total_amount","CustomerID"]], left_on="CustomerID",right_on="CustomerID")

In [129]:
orders_catclient.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Client_status,total_amount
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred,5391.21
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,5391.21
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred,5391.21
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,5391.21
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,5391.21


In [130]:
valuable_clients = orders_catclient[(orders_catclient["Client_status"] == "VIP")| (orders_catclient["Client_status"] == "Preferred")].copy()
valuable_clients.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Client_status,total_amount
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred,5391.21
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,5391.21
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred,5391.21
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,5391.21
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred,5391.21


In [131]:
vippref_pcountry = valuable_clients.groupby(["Country"]).count().iloc[:,0]
vippref_pcountry.sort_values(ascending=False)

Country
United Kingdom     221635
Germany              7349
EIRE                 7238
France               6301
Netherlands          2080
Spain                1569
Belgium              1557
Switzerland          1370
Portugal             1093
Norway               1028
Australia            1028
Channel Islands       589
Italy                 507
Finland               504
Cyprus                451
Sweden                273
Japan                 272
Singapore             222
Denmark               217
Israel                214
Iceland               182
Austria               158
Poland                149
Canada                135
Malta                  67
Lebanon                45
Greece                 32
Name: InvoiceNo, dtype: int64

In [132]:
only_vip = valuable_clients[valuable_clients["Client_status"]=="VIP"]
vip_pcountry = only_vip.groupby(["Country"]).count().iloc[:,0]
vip_pcountry.sort_values(ascending=False)

Country
United Kingdom     84185
EIRE                7077
France              3290
Germany             3127
Netherlands         2080
Australia            898
Portugal             681
Switzerland          594
Spain                511
Norway               420
Channel Islands      364
Finland              294
Cyprus               248
Singapore            222
Japan                205
Sweden               198
Belgium               54
Denmark               36
Name: InvoiceNo, dtype: int64

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# your code here

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [135]:
# your code here
only_vip = valuable_clients[valuable_clients["Client_status"]=="VIP"]
vip_pcountry = only_vip.groupby(["Country"]).count().iloc[:,0]
vip_pcountry.sort_values(ascending=False).reset_index().iloc[[0],:]

,Country,InvoiceNo
0,United Kingdom,84185


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [138]:
# your code here
vippref_pcountry = valuable_clients.groupby(["Country"]).count().iloc[:,0]
vippref_pcountry.sort_values(ascending=False).reset_index().iloc[[0],:]

,Country,InvoiceNo
0,United Kingdom,221635
